In [ ]:
#### -----------
#### Use the scanpy.yml environment
#### -----------

import os
import re 
import numpy as np 
import pandas as pd 
import anndata as ad 
import scanpy as sc 
import seaborn as sns
import matplotlib.pyplot as plt 
from matplotlib.patches import Patch
import plotly.express as px
import plotly.graph_objects as go

In [ ]:
# core ID 
CORE = 'E06'

# read input data 
ddir = '/Users/watsocam/OHSU/analysis/TMA_integation/'
metadata = pd.read_csv(os.path.join(ddir,'metadata/metadata_clean.csv'))

### Immune panel (all)

In [ ]:
# read input data 
all_cores_df = pd.read_csv(os.path.join(ddir, 'labeled_tables/immune_all_labeled.csv'))

# subset to individual core (E06)
df = all_cores_df[all_cores_df['core'] == 'E06']
df = df.reset_index(drop=True)
print(len(df))
print(df.columns)

In [ ]:
# create anndata object 
obs_cols = ['CellID', 'core', 'DNA_1', 'cluster', 'cross_cluster']
var_cols = [x for x in list(df.columns) if x not in obs_cols]
adata = ad.AnnData(X = df[var_cols], obs = df[obs_cols])
adata.var_names

In [ ]:
# create an anndata with a reduced marker set 
markers = ['CD3', 'GRZB', 'Ki67', 'PanCK', 'CD45', 'CD68', 'CD3d', 'CD8a',
       'CD163', 'aSMA', 'CD14', 'CD4', 'FOXP3', 'CD11b', 'CD20', 'MHC_II_DPB1']
       
adata_reduced = adata.copy()
adata_reduced = adata_reduced[:,markers]

In [ ]:
# run scanpy tools on the reduced marker set data to help assign cell types 
sc.pp.neighbors(adata_reduced)
sc.tl.umap(adata_reduced)
sc.tl.leiden(adata_reduced, resolution = 2)

In [ ]:
# generate a heatmap of mean marker intensities by leiden cluster 
heatmap_df = adata_reduced.to_df()
heatmap_df['leiden'] = adata_reduced.obs['leiden']
heatmap_df = heatmap_df.groupby('leiden').mean()

In [ ]:
g = sns.clustermap(heatmap_df, z_score=1, cmap = 'Spectral_r', vmin = -2, vmax = 2)
g.fig.set_facecolor('white')
plt.savefig('figures/E06_immune_heatmap.png', dpi = 500)

In [ ]:
# SAVE STUFF! 
df['leiden'] = list(adata_reduced.obs['leiden'])
df.to_csv('data/E06_immune_indiv_leiden.csv', index=False)

### Tumor panel (all)

In [ ]:
# read input data 
all_cores_df = pd.read_csv(os.path.join(ddir, 'labeled_tables/tumor_all_labeled.csv'))

# subset to individual core (E06)
df = all_cores_df[all_cores_df['core'] == 'E06']
df = df.reset_index(drop=True)
print(len(df))
print(df.columns)

In [ ]:
# create anndata object 
obs_cols = ['CellID', 'core', 'DNA_1', 'cluster', 'cross_cluster']
var_cols = [x for x in list(df.columns) if x not in obs_cols]
adata = ad.AnnData(X = df[var_cols], obs = df[obs_cols])
adata.var_names

In [ ]:
# create an anndata with a reduced marker set 
markers = ['CD3',  'Vimentin', 'aSMA', 'Ecad', 'Ki67', 'CD45', 
           'CK14', 'CK19', 'CK17', 'PCNA', 'PanCK', 'CD31']
       
adata_reduced = adata.copy()
adata_reduced = adata_reduced[:,markers]

In [ ]:
# run scanpy tools on the reduced marker set data to help assign cell types 
sc.pp.neighbors(adata_reduced)
sc.tl.umap(adata_reduced)
sc.tl.leiden(adata_reduced, resolution = 2)

In [ ]:
# generate a heatmap of mean marker intensities by leiden cluster 
heatmap_df = adata_reduced.to_df()
heatmap_df['leiden'] = adata_reduced.obs['leiden']
heatmap_df = heatmap_df.groupby('leiden').mean()

In [ ]:
g = sns.clustermap(heatmap_df, z_score=1, cmap = 'Spectral_r', vmin = -2, vmax = 2)
g.fig.set_facecolor('white')
plt.savefig('figures/E06_tumor_heatmap.png', dpi = 500)

In [ ]:
# SAVE STUFF! 
df['leiden'] = list(adata_reduced.obs['leiden'])
df.to_csv('data/E06_tumor_indiv_leiden.csv', index=False)

### Tracked cells

In [ ]:
# read input data 
all_cores_df = pd.read_parquet(os.path.join(ddir, 'labeled_tables/all_tracked_100NN_leiden.parquet'))

# subset to individual core (E06)
df = all_cores_df[all_cores_df['core'] == 'E06']
df = df.reset_index(drop=True)
print(len(df))
print(df.columns)

In [ ]:
# create anndata object 
obs_cols = ['tumor_CellID', 'core', 'tumor_DNA_1', 
            'tumor_cluster', 'immune_CellID', 'immune_DNA_1',
            'immune_cluster','combined_cluster']
var_cols = [x for x in list(df.columns) if x not in obs_cols]
adata = ad.AnnData(X = df[var_cols], obs = df[obs_cols])
adata.var_names

In [ ]:
# create an anndata with a reduced marker set 

# immune 
# markers = ['CD3', 'GRZB', 'Ki67', 'PanCK', 'CD45', 'CD68', 'CD3d', 'CD8a',
#        'CD163', 'aSMA', 'CD14', 'CD4', 'FOXP3', 'CD11b', 'CD20', 'MHC_II_DPB1']

# tumor
# markers = ['CD3',  'Vimentin', 'aSMA', 'Ecad', 'Ki67', 'CD45', 
#            'CK14', 'CK19', 'CK17', 'PCNA', 'PanCK', 'CD31']

markers = ['tumor_CD3', 'tumor_Vimentin', 'tumor_aSMA', 'tumor_Ecad',
           'tumor_Ki67', 'tumor_CD45', 'tumor_CK14', 'tumor_CK19', 'tumor_CK17',
           'tumor_PCNA', 'tumor_PanCK', 'tumor_CD31',
           'immune_CD3', 'immune_GRZB', 'immune_Ki67',
           'immune_PanCK', 'immune_CD45', 'immune_CD68', 'immune_CD3d',
           'immune_CD8a', 'immune_CD163', 'immune_aSMA', 'immune_CD14',
           'immune_CD4', 'immune_FOXP3', 'immune_CD11b',
           'immune_CD20', 'immune_MHC_II_DPB1'
]
       
adata_reduced = adata.copy()
adata_reduced = adata_reduced[:,markers]

In [ ]:
# run scanpy tools on the reduced marker set data to help assign cell types 
sc.pp.neighbors(adata_reduced)
sc.tl.umap(adata_reduced)
sc.tl.leiden(adata_reduced, resolution = 2)

In [ ]:
# generate a heatmap of mean marker intensities by leiden cluster 
heatmap_df = adata_reduced.to_df()
heatmap_df['leiden'] = adata_reduced.obs['leiden']
heatmap_df = heatmap_df.groupby('leiden').mean()

In [ ]:
heatmap_df.to_csv('data/E06_tracked_heatmap_df.csv' ,index=True)

In [ ]:
g = sns.clustermap(heatmap_df, z_score=1, cmap = 'Spectral_r', vmin = -2, vmax = 2)
g.fig.set_facecolor('white')
plt.savefig('figures/E06_tracked_heatmap.png', dpi = 500)

In [ ]:
# SAVE STUFF! 
df['leiden'] = list(adata_reduced.obs['leiden'])
df.to_csv('data/E06_tracked_indiv_leiden.csv', index=False)